In [ ]:
import wec_as_multiport as wam
import wecopttool as wot
import capytaine as cpy
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd

tex_fonts = {
    # Use LaTeX to write all text
    "text.usetex": True,
    "font.family": "serif",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 10,
    "font.size": 10,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "grid.linestyle": '--',
    "grid.linewidth": '0.1',
    "grid.alpha": 0.25,
    "xtick.major.width": 0.5,
    "ytick.major.width": 0.5,
    "xtick.minor.width": 0.25,
    "ytick.minor.width": 0.25,
}

plt.rcParams.update(tex_fonts)

gfx_path = os.path.join('gfx')
data_path = os.path.join('data')
base_name = 'wec_as_multiport_'

rho = 1e3
g = 9.81

In [ ]:
f1 = 0.025
nfreq = 60
freq = wot.frequency(f1, nfreq, False)  # False -> no zero frequency

wb = wot.geom.WaveBot()  # use standard dimensions
mesh_size_factor = 0.5  # 1.0 for default, smaller to refine mesh
mesh = wb.mesh(mesh_size_factor)
fb = cpy.FloatingBody.from_meshio(mesh, name="WaveBot")
fb.add_translation_dof(name="Heave")

# BEM
## Run Capytaine if needed

In [ ]:
bem_data_fname = os.path.join(data_path,'wec_as_multiport.nc')

if os.path.isfile(bem_data_fname):
    print("Found existing BEM file, loading")
    bem_data = wot.read_netcdf(bem_data_fname)
else:
    bem_data = wot.run_bem(fb, freq)
    bem_data = bem_data.assign_coords(
        freq=("omega", bem_data['omega'].values/2/np.pi))
    bem_data['freq'].attrs['long_name'] = 'Frequency'
    bem_data['freq'].attrs['units'] = 'Hz'
    bem_data['excitation_force'] = bem_data['diffraction_force'] + bem_data['Froude_Krylov_force']
    bem_data = wot.add_linear_friction(bem_data)
    wot.write_netcdf(bem_data_fname, bem_data)

## Clean up with MATLAB function 

In [ ]:
clean_bem_data_fname = os.path.join(data_path,'wec_as_multiport_clean.nc')

if os.path.isfile(clean_bem_data_fname):
    print("Found existing clean BEM file, loading")
    clean_bem_data = wot.read_netcdf(clean_bem_data_fname)
else:
    import matlab.engine
    eng = matlab.engine.start_matlab("-noFigureWindows")

    eng.cd(os.path.join(os.path.abspath(""),"matlab"))
    clean_hydro = eng.fix_bem_spikes(os.path.join(os.path.abspath(""),data_path,'wec_as_multiport.nc'))
    clean_hydro = eng.fix_bem_spikes(os.path.join(os.path.abspath(""),data_path,'wec_as_multiport.nc'))
    eng.exit()
    
    wanted_keys = ['w','fk_im', 'fk_re', 'A', 'B', 'sc_re', 'sc_im', 'ex_re', 'ex_im']
    a = dict((k, clean_hydro[k]) for k in wanted_keys if k in clean_hydro)
    a = {k: np.array(v) for k, v in a.items()}

    clean_bem_data = bem_data.copy()

    clean_bem_data['added_mass'] = (
        ['omega', 'radiating_dof', 'influenced_dof'],
        np.expand_dims((a['A']*rho).transpose(2, 0, 1)[:, 2, 2], axis=[1, 2]))

    clean_bem_data['radiation_damping'] = (
        ['omega', 'radiating_dof', 'influenced_dof'],
        np.expand_dims((a['B']*rho*a['w']).transpose(2, 0, 1)[:, 2, 2], axis=[1, 2]))

    clean_bem_data['Froude_Krylov_force'] = (
        ['omega', 'wave_direction', 'influenced_dof'],
        np.expand_dims(((a['fk_re']+1j*a['fk_im'])*rho*g).transpose(2,
                    0, 1)[:, 2, 0], axis=[1, 2]))

    clean_bem_data['diffraction_force'] = (
        ['omega', 'wave_direction', 'influenced_dof'],
        np.expand_dims(((a['sc_re']+1j*a['sc_im'])*rho*g).transpose(2,
                    0, 1)[:, 2, 0], axis=[1, 2]))

    clean_bem_data['excitation_force'] = (
        ['omega', 'wave_direction', 'influenced_dof'],
        np.expand_dims(((a['ex_re']+1j*a['ex_im'])*rho*g).transpose(2,
                    0, 1)[:, 2, 0], axis=[1, 2]))
    
    wot.write_netcdf(clean_bem_data_fname, clean_bem_data)


In [ ]:
wec = wam.WEC(omega=clean_bem_data['omega'].values,
            N=12.4666,
            Kt=6.1745,
            Rw=0.5,
            Lw=0,
            Jd=2,
            Bd=1,
            Kd=0,
            Zi=np.squeeze(wot.hydrodynamic_impedance(clean_bem_data)).values,
            Hexc=np.squeeze(clean_bem_data['excitation_force'].values))

In [ ]:
fig, ax = plt.subplots(nrows=2,
                       sharex=True)

ax[0].plot(wec.freq, 20*np.log10(np.abs(wec.Zi)), color='k')
ax[0].set_ylabel('$|Z_i|$ [dB]')

ax[1].plot(wec.freq, np.angle(wec.Zi), color='k')
ax[1].set_ylabel('$\\angle{Z_i}$ [rad]')

ax[-1].set_xlabel('Frequency [Hz]')

for axi in ax:
    axi.set_xscale('log')
    axi.autoscale(enable=True, axis='x', tight=True)
    axi.grid(which='major', axis='x')
    axi.set_title('')
    axi.label_outer()

In [ ]:
fig, ax = plt.subplots(nrows=2,
                       sharex=True)

ax[0].plot(wec.freq, np.real(wec.Zi), color='k')
ax[0].set_ylabel('$\\Re\{Z_i\}$ [XX]')

ax[1].plot(wec.freq, np.imag(wec.Zi), color='k')
ax[1].set_ylabel('$\\Im\{Z_i\}$ [XX]')

for axi in ax:
    axi.set_xscale('log')
    axi.autoscale(enable=True, axis='x', tight=True)
    axi.grid(which='both', axis='both')
    axi.set_title('')
    axi.label_outer()
    
ax[-1].set_xlabel('Frequency [Hz]')

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex=True)

ax[0].plot(wec.freq, np.abs(wec.Hexc), color='k')
ax[1].plot(wec.freq, np.angle(wec.Hexc), color='k')

ax[0].set_ylabel('$| H_{exc} | $ [N/m]')
ax[1].set_ylabel('$\\angle{H_{exc}}$ [rad]')
ax[-1].set_xlabel('Frequency [Hz]')

for axi in ax:
    axi.set_xscale('log')
    axi.autoscale(enable=True, axis='x', tight=True)
    axi.grid(which='both', axis='both')
    axi.set_title('')
    axi.label_outer()

# Contoller design

## Controller impedances

In [ ]:
fig, ax = plt.subplots(nrows=3,
                       sharex=True,
                       figsize=wam.figsize(hf=1.5))

ax[0].plot(wec.freq, 20*np.log10(np.abs(wec.Zl_opt_mech)),
           color='k', 
           ls='--', 
           label='$Z_\ell \\vert_{Z_{\ell m} = Z_i^*}$')
ax[0].plot(wec.freq, 20*np.log10(np.abs(wec.Zl_opt)),
           color='k', 
           ls='-',
           label='$Z_\ell = Z_{\\mathrm{out}}^*$')

ax[1].plot(wec.freq, np.angle(wec.Zl_opt_mech),
           color='k', 
           ls='--', 
           label='$Z_\ell \\vert_{Z_{\ell m} = Z_i^*}$')
ax[1].plot(wec.freq, np.angle(wec.Zl_opt),
           color='k', 
           ls='-',
           label='$Z_\ell = Z_{\\mathrm{out}}^*$')

ax[2].plot(wec.freq, wec.transducer_power_gain(Zl=wec.Zl_opt), 
        ls='-', 
        color = 'k', 
        label='$Z_\ell = Z_{\\mathrm{out}}^*$')

ax[2].plot(wec.freq,wec.transducer_power_gain(Zl=wec.Zl_opt_mech), 
        ls='--', 
        color = 'k', 
        label='$Z_\ell \\vert_{Z_{\ell m} = Z_i^*}$')

for axi in ax:
    axi.set_xscale('log')
    axi.autoscale(enable=True, axis='x', tight=True)
    axi.grid(which='both', axis='both')
    axi.set_title('')
    axi.label_outer()
    axi.xaxis.set_minor_formatter(NullFormatter())
    axi.set_xlim([1e-1,1])
    axi.spines[['top','bottom','left','right']].set_linewidth(0.5)
    
ax[2].set_yscale('linear')
ax[2].set_ylim([-1,1])
    
ax[0].legend()
ax[-1].set_xlabel('Frequency [Hz]')

ax[0].set_ylabel('$|Z|$ [dB]')
ax[1].set_ylabel('$\\angle{Z}$ [rad]')
ax[2].set_ylabel('Trans. gain')

fig.savefig(os.path.join(gfx_path,base_name + 'load_impedance_for_mech_power_Bode.pdf'), bbox_inches = "tight")

In [ ]:
fig, ax = plt.subplots(nrows=3,
                       sharex=True,
                       figsize=wam.figsize(hf=1.5))

ax[0].plot(wec.freq, np.real(np.abs(wec.Zl_opt_mech)),
           color='k', 
           ls='--', 
           label='$Z_\ell \\vert_{Z_{\ell m} = Z_i^*}$')
ax[0].plot(wec.freq, np.real(np.abs(wec.Zl_opt)),
           color='k', 
           ls='-',
           label='$Z_\ell = Z_{\\mathrm{out}}^*$')

ax[1].plot(wec.freq, np.imag(wec.Zl_opt_mech),
           color='k', 
           ls='--', 
           label='$Z_\ell \\vert_{Z_{\ell m} = Z_i^*}$')
ax[1].plot(wec.freq, np.imag(wec.Zl_opt),
           color='k', 
           ls='-',
           label='$Z_\ell = Z_{\\mathrm{out}}^*$')

ax[2].plot(wec.freq, wec.transducer_power_gain(Zl=wec.Zl_opt), 
        ls='-', 
        color = 'k', 
        label='$Z_\ell = Z_{\\mathrm{out}}^*$')

ax[2].plot(wec.freq,wec.transducer_power_gain(Zl=wec.Zl_opt_mech), 
        ls='--', 
        color = 'k', 
        label='$Z_\ell \\vert_{Z_{\ell m} = Z_i^*}$')

for axi in ax:
    axi.set_xscale('log')
    axi.autoscale(enable=True, axis='x', tight=True)
    axi.grid(which='both', axis='both')
    axi.set_title('')
    axi.label_outer()
    axi.xaxis.set_minor_formatter(NullFormatter())
    axi.set_xlim([1e-1,1])
    axi.spines[['top','bottom','left','right']].set_linewidth(0.5)
    
ax[2].set_yscale('linear')
ax[2].set_ylim([-1,1])
    
ax[0].legend()
ax[-1].set_xlabel('Frequency [Hz]')

ax[0].set_ylabel('$\\Re \{ Z \}$ [V/A]')
ax[1].set_ylabel('$\\Im \{ Z \}$ [V/A]')
ax[2].set_ylabel('Trans. gain')

fig.savefig(os.path.join(gfx_path,base_name + 'load_impedance_for_mech_power_real_imag.pdf'), bbox_inches = "tight")

In [ ]:
wec.freq

In [ ]:
mask = (wec.freq >= 0.1) & (wec.freq <= 1)

fig, ax = plt.subplots(nrows=3,sharex=True,figsize=wam.figsize(hf=1.5))

ax[0].plot(wec.freq[mask], np.real(wec.Zl_opt[mask]), 
           color='k', ls='-')
ax[1].plot(wec.freq[mask], np.imag(wec.Zl_opt[mask]), 
           color='k', ls='-')

ax[2].plot(wec.freq[mask], wec.transducer_power_gain()[mask], color='k')



# design_freqs = [0.55, 0.42, 0.35]
design_freqs = [0.4, 0.55, 0.65]
# colors = plt.get_cmap('Paired', len(design_freqs)).colors
colors = ['C0','C1','C2']

for design_freq, color in zip(design_freqs,colors):

    kp, ki = wec.pi_opt(design_freq)
    print(f'kp, ki: ({kp:.2f}, {ki:.2f})')
    C = wec.pid_controller(kp=kp, ki=ki)
    Zl_C = wec.Zl_C(C)

    ax[0].plot(wec.freq[mask], np.real(Zl_C[mask]), 
            color=color, ls='-', lw=1)
    ax[1].plot(wec.freq[mask], np.imag(Zl_C[mask]), 
            color=color, ls='-', lw=1)

    ax[2].fill_between(wec.freq[mask], 
               wec.transducer_power_gain(Zl=Zl_C)[mask],
               color=color, alpha=0.25,
               label=f'$f_d={design_freq}$\,Hz')
    
    
    # print(np.trapz(wec.transducer_power_gain(Zl=Zl_C), wec.freq))
    
    for axi in ax:
        axi.axvline(design_freq, color=color, ls='--', lw=1, alpha=0.5)

# ax[2].set_yscale('log')

for axi in ax:
    axi.set_xscale('log')
    axi.autoscale(enable=True, axis='x', tight=True)
    axi.grid(which='both', axis='both')
    axi.set_title('')
    axi.label_outer()
    axi.xaxis.set_minor_formatter(NullFormatter())
    axi.set_xlim([1e-1,1])
    axi.spines[['top','bottom','left','right']].set_linewidth(0.5)
    
ax[1].set_ylim(bottom=-4)
ax[2].set_ylim(bottom=0)

ax[-1].set_xlabel('Frequency [Hz]')

ax[0].set_ylabel('$\\Re \{ Z \}$ [V/A]')
ax[1].set_ylabel('$\\Im \{ Z \}$ [V/A]')
ax[2].set_ylabel('Trans.\ngain [ ]')

ax[-1].legend()
ax[-1].set_ylim(top=1)

fig.savefig(os.path.join(gfx_path,base_name + 'pi_controllers_real_imag.pdf'), bbox_inches = "tight")

## Power output

In [ ]:
waves = wot.waves.regular_wave(f1=wec.f1, nfreq=wec.nfreq, freq=0.4, amplitude=0.25)
Fexc = wec.Fexc(waves=waves.squeeze().values)
Fth = wec.F_Thevenin(Fexc=Fexc)

fig, ax = plt.subplots(nrows=3, sharex=True)

ax[0].plot(np.abs(Fth), label='Thevenin')
ax[0].plot(np.abs(Fexc), label='Regular')
ax[0].set_ylabel('Excitation')

ax[1].plot(np.real(wec.Z_Thevenin))
ax[1].plot(np.real(wec.Zi))
ax[1].set_ylabel('Real impedance')

ax[2].plot(wec.max_active_power(Fexc=Fexc))
ax[2].plot(wec.max_active_power_mech(Fexc=Fexc))
ax[2].set_ylabel('Max power')

ax[0].legend()

In [ ]:
# wot.waves.jonswap_spectrum(freq=wec.freq, fp=0.4, hs=1, gamma=1)
# wot.waves.long_crested_wave()

fp_list = [0.4, 0.5, 0.6]
pow_list = []

# wec.Rw = 0?

for fp in fp_list:
    # efth = wot.waves.omnidirectional_spectrum(wec.f1, 
    #                                         wec.nfrq, 
    #                                         lambda f: wot.waves.pierson_moskowitz_spectrum(f, fp, 0.2), 
    #                                         "Pierson-Moskowitz")
    # waves = wot.waves.long_crested_wave(efth, 1)
    waves = wot.waves.regular_wave(f1=wec.f1, nfreq=wec.nfreq, freq=fp, amplitude=0.25)
    Fexc = wec.Fexc(waves=waves.squeeze().values)
    pow = np.sum(wec.active_power(Fexc=Fexc, Zl=wec.Zl_opt))
    pow_list.append(pow)
    # print(pow)
    
df = pd.DataFrame(pow_list, fp_list, 
                  columns=['Elec. power [W]'])
df
# print(df.transpose().to_latex(float_format='%.1f'))

# Compare different drive trains

In [ ]:
def make_design_var_plots(wec, var_name, values, colors):
    wec_list = []
    for val in values:
        wec_tmp = wec.copy()
        wec_tmp.__setattr__(var_name, val)
        wec_list.append(wec_tmp)

    mask = (wec.freq >= 0.1) & (wec.freq <= 1)

    fig, ax = plt.subplots(nrows=2,
                           ncols=2,
                           sharex=True,
                           figsize=wam.figsize())

    for wec1, color in zip(wec_list, colors):

        leg_string = '$Z_{\mathrm{in}}($' + f'{var_name}' + '$=$' + \
            f'{wec1.__getattribute__(var_name)})'

        ax[0, 0].plot(wec1.freq[mask], np.real(wec1.Zin()[mask]),
                      color=color,
                      label=leg_string)
        ax[1, 0].plot(wec1.freq[mask], np.imag(wec1.Zin()[mask]),
                      color=color,
                      label=leg_string)

        ax[0, 1].plot(wec1.freq[mask], np.real(wec1.Zout[mask]),
                      color=color,
                      label=leg_string)
        ax[1, 1].plot(wec1.freq[mask], np.imag(wec1.Zout[mask]),
                      color=color,
                      label=leg_string)

        # ax2.plot(wec1.freq[mask],
        #         wec1.transducer_power_gain(Zl=wec.Zl_opt)[mask],
        #         color=color,
        #         label=leg_string)

    ax[0, 0].plot(wec.freq[mask], np.real(np.conj(wec.Zi[mask])),
                  color='k',
                  ls='--',
                  label='$Z_i^*$')
    ax[1, 0].plot(wec.freq[mask], np.imag(np.conj(wec.Zi[mask])),
                  color='k',
                  ls='--',
                  label='$Z_i^*$')

    for axi in ax.flatten():
        axi.set_xscale('log')
        axi.autoscale(enable=True, axis='x', tight=True)
        axi.grid(which='both', axis='both')
        axi.set_title('')
        axi.xaxis.set_minor_formatter(NullFormatter())
        axi.spines[['top','bottom','left','right']].set_linewidth(0.5)

    ax[0, 0].set_ylabel('$\\Re\{Z\}$')
    ax[1, 0].set_ylabel('$\\Im\{Z\}$')

    # ax[1, 0].legend(ncol=2, fontsize='x-small')

    ax[0][0].set_title('PTO input', fontsize=10)
    ax[0][1].set_title('PTO output', fontsize=10)
        
    fig.supxlabel('Frequency [Hz]',
                  fontsize=10)
    
    fig.tight_layout(pad=0.1)

    return wec_list, fig


wec_designs = {
    'Kd': [],
    'Jd': []
}

my_colors = [
    plt.cm.Reds(np.linspace(0.4, 0.75, 3)),
    plt.cm.Blues(np.linspace(0.4, 0.75, 3)),
]

wec_designs['Kd'], fig = make_design_var_plots(wec,
                                          "Kd",
                                          np.linspace(0, -100, 3),
                                          my_colors[0],
                                          )

fig.get_axes()[2].set_ylim(top=2e4)

an = fig.get_axes()[0].annotate('Stronger\nspring', 
                           xy=(0.16, 1.6e3), 
                           xytext=(0.49, 4.1e3),
                           color=my_colors[0][-1],
                           fontsize=8,
                           fontweight='heavy',
                           ha='center',
                           arrowprops=dict(
                               arrowstyle="->",
                               connectionstyle="arc3,rad=0.3",
                               color=my_colors[0][-1],
                               ))

fig.get_axes()[0].text(0.11, 8e2, '$Z_i^*$')

fig.savefig(os.path.join(gfx_path,base_name + 'in_and_out_impedances_spring.pdf'), bbox_inches = "tight")

wec_designs['Jd'], fig = make_design_var_plots(wec,
                                          "Jd",
                                          np.linspace(2, 20, 3),
                                          my_colors[1],
                                          )

fig.get_axes()[2].set_ylim(top=2e4)

fig.get_axes()[2].annotate('More\ninertia', 
                           xy=(0.2, 1.1e4), 
                           xytext=(0.35, -7e3),
                           color=my_colors[1][-1],
                           fontsize=8,
                           fontweight='heavy',
                           ha='center',
                           arrowprops=dict(
                               arrowstyle="->",
                               connectionstyle="arc3,rad=0.3",
                               color=my_colors[1][-1],
                               ))

fig.get_axes()[0].text(0.11, 8e2, '$Z_i^*$')

fig.savefig(os.path.join(gfx_path,base_name + 'in_and_out_impedances_inertia.pdf'), bbox_inches = "tight")

fig, ax = plt.subplots(figsize=wam.figsize())

for (key, wecs), colors, ls in zip(wec_designs.items(), my_colors, ['-','--']):
        for wec1, color in zip(wecs, colors):
                ax.plot(wec1.freq, wec1.transducer_power_gain(Zl=wec1.Zl_opt),
                        ls=ls,
                        color=color,
                        )
                
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Transducer gain [ ]')
ax.spines[['right', 'top']].set_visible(False)
ax.set_ylim([0,1])
ax.autoscale(enable=True, axis='x', tight=True)
ax.set_xscale('log')
ax.set_xlim([1e-1, 1])
ax.xaxis.set_minor_formatter(NullFormatter())

# fig.savefig(os.path.join(gfx_path,base_name + 'spring_inertia_transducer_gains.pdf'))